From: https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

In [1]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

header = ["color", "diameter", "label"]

In [2]:
def unique_vals(data, col):
    return set([row[col] for row in data])

unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [3]:
def class_counts(data):
    counts = {}
    for row in data:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [4]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

is_numeric("Red")

False

In [5]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

Question(0, 3)

Is color >= 3?

In [6]:
q = Question(0, 'Green')
example = training_data[0]
q.match(example)

True

In [7]:
def partition(data, question):
    true_rows, false_rows = [], []
    for row in data:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

true_rows, false_rows = partition(training_data, Question(0, 'Red'))
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [8]:
def gini(data):
    counts = class_counts(data)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(data))
        impurity -= prob_of_label ** 2
    return impurity

no_mixing = [['Apple'], ['Apple']]
gini(no_mixing)

0.0

In [9]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [10]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [11]:
def find_best_split(data):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(data)
    n_features = len(data[0]) - 1

    for col in range(n_features):
        values = unique_vals(data, col)
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(data, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [12]:
best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

In [13]:
class Leaf:
    def __init__(self, data):
        self.predictions = class_counts(data)

In [14]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
def build_tree(data):
    gain, question = find_best_split(data)

    if gain == 0:
        return Leaf(data)

    true_rows, false_rows = partition(data, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [16]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    print(spacing + str(node.question))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [17]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [18]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
classify(training_data[0], my_tree)

{'Apple': 1}

In [19]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [20]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
